# Projet sur la production d'une **Etude de marché**
<br/>

## Partie I : Préparation et nettoyage des données
<br/>

Dans ce notebook nous ferons essentiellement du nettoyage de données. Ces données pour la plupart sont exportées depuis la base de données de la FAO et les distances entre les autres pays et la France provoviennent du [github de rahlbot](https://github.com/rahulbot/distances-between-countries).

In [1]:
# Importons les librairies pour le traitement des données
import numpy as np
import pandas as pd

In [2]:
# Importation des données

pop=pd.read_csv('dataset_p5/donnees/population_2017_2018.csv')# données sur la population de chaque pays
ISO=pd.read_csv('dataset_p5/donnees/infos_ISO.csv') # données dur le code ISO3 de chaque pays
dispo_alim=pd.read_csv('dataset_p5/donnees/dispo_2018.csv') # données sur la disponibilité alimentaire des pays
prix_poulet=pd.read_csv('dataset_p5/donnees/prix_poulet_annuel.csv')# données coût de production du poulet dans les pays
pib= pd.read_csv('dataset_p5/donnees/pib_par_hab.csv') # données sur l'activité économique des pays
volailles= pd.read_csv('dataset_p5/donnees/donnee_volailles_2018.csv') # données sur la dispnibilité intérieure en volaille
data_continent= pd.read_csv('dataset_p5/donnees/data_continent.csv', index_col=[0]) # données continent de chaque pays
distances_france_km = pd.read_csv('dataset_p5/new_data/distance_france_km.csv')


In [3]:
# affichons la table population
pop.head(5)

,Code Domaine,Domaine,Code zone (FAO),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
0,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,36296.113,X,Sources internationales sûres,NaN
1,OA,Séries temporelles annuelles,2,Afghanistan,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,37171.921,X,Sources internationales sûres,NaN
2,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,57009.756,X,Sources internationales sûres,NaN
3,OA,Séries temporelles annuelles,202,Afrique du Sud,511,Population totale,3010,Population-Estimations,2018,2018,1000 personnes,57792.518,X,Sources internationales sûres,NaN
4,OA,Séries temporelles annuelles,3,Albanie,511,Population totale,3010,Population-Estimations,2017,2017,1000 personnes,2884.169,X,Sources internationales sûres,NaN


In [4]:
# statistique élémentaire
pop.describe()

,Code zone (FAO),Code Élément,Code Produit,Code année,Année,Valeur
count,474.000000,474.0,474.0,474.000000,474.000000,4.740000e+02
mean,135.805907,511.0,3010.0,2017.500000,2017.500000,3.816773e+04
std,79.051916,0.0,0.0,0.500528,0.500528,1.611099e+05
min,1.000000,511.0,3010.0,2017.000000,2017.000000,7.930000e-01
25%,69.000000,511.0,3010.0,2017.000000,2017.000000,3.837125e+02
50%,134.000000,511.0,3010.0,2017.500000,2017.500000,5.317144e+03
75%,199.000000,511.0,3010.0,2018.000000,2018.000000,1.972709e+04
max,351.000000,511.0,3010.0,2018.000000,2018.000000,1.459378e+06


In [5]:
# Restriction au nombre de colonne souhaité 
pop=pop[['Code zone (FAO)','Zone','Année','Valeur']]

In [6]:
# convertissons la valeur de la poppulation
pop['Valeur']=pop['Valeur']*1000

### ajoutons les régions à  nos données

In [7]:
# mettons sur une même table les informations des pays, population et region
pop=pd.merge(pop,data_continent, on = 'Code zone (FAO)', how = 'left')

In [8]:
# affichons la table contenant les données ISO3
ISO.sample()

,Code Pays,Pays,Code M49,Code ISO2,Code ISO3,Année de début,Año de inicio
58,18,Bhoutan,64.0,BT,BTN,NaN,NaN


In [9]:
#sélection de colonnes
ISO=ISO[['Code Pays','Code ISO3']]
# renommage des colonnes
ISO.columns=['Code zone (FAO)','Code ISO3']

### ajoutons le Code ISO3 à   nos données

In [10]:
pop = pd.merge(pop,ISO,on ='Code zone (FAO)', how = 'left')

In [11]:
pop.sample()

,Code zone (FAO),Zone,Année,Valeur,Continent,Code ISO3
176,93,Haïti,2017,10982366.0,Amerique,HTI


In [12]:
# affichons les années sur les colonnes en faisant un pivot
tab_pop=pd.pivot_table(pop,index=['Code zone (FAO)','Zone','Continent','Code ISO3'],columns='Année',values='Valeur')
tab_pop.reset_index(inplace=True)
tab_pop.sample()

Année,Code zone (FAO),Zone,Continent,Code ISO3,2017,2018
35,44,Colombie,Amerique,COL,48909839.0,49661048.0


In [13]:
tab_pop.shape

(235, 6)

In [14]:
# renommons nos colonnes
tab_pop.columns=['Code_pays','Pays', 'Région','Code ISO3','pop_17', 'pop_18']

In [15]:
tab_pop.sample()

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18
197,222,Tunisie,Afrique,TUN,11433443.0,11565201.0


## Travaillons sur les disponibilités alimentaires

In [16]:
#affichons notre table
dispo_alim.sample()

,Code Domaine,Domaine,Code zone (FAO),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
176,FBS,Bilans Alimentaires (2014-),72,Djibouti,664,Disponibilité alimentaire (Kcal/personne/jour),2901,Total General,2018,2018,Kcal/personne/jour,2694.0,Fc,Donnée calculée


In [17]:
# choisissons les colonnes dont nous avons besoin
dispo_alim=dispo_alim[['Code zone (FAO)','Élément','Produit','Valeur']]
dispo_alim.sample()

,Code zone (FAO),Élément,Produit,Valeur
347,113,Disponibilité de protéines en quantité (g/pers...,Produits Animaux,33.91


Nous allons créer une colonne qui regroupe les éléments et les produits  pour différencier le général du type animal:
nous n'aurons plusque 4 éléments différents pour cette colonne
* les disponibilites alimentaires-total général,
* les disponibilités de protéine en quantité-total général,
* les disponibités alimentaires -produits animaux,
* les disponibilités de protéine en quantité-produits animaux 

In [18]:
# Concatenons les colonnes éléments et produits

cols=["Élément","Produit"]

dispo_alim['Element_prod']=dispo_alim[cols].sum(1)
dispo_alim = dispo_alim.drop(cols, 1)
dispo_alim.sample()

,Code zone (FAO),Valeur,Element_prod
225,66,71.14,Disponibilité de protéines en quantité (g/pers...


In [19]:
dispo_alim.shape

(692, 3)

In [20]:
# mettons nos disponibilites en colonnes par un pivot

dispo_alim = dispo_alim.pivot_table(index='Code zone (FAO)', columns='Element_prod',values='Valeur')
dispo_alim.reset_index(inplace = True)
# Renommons nos colonnes
dispo_alim.columns=['Code_pays','Dispo_alim_Animaux','Dispo_alim_general','Dispo_protéines_Animaux','Dispo_protéines_general']
dispo_alim


,Code_pays,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general
0,1,795.0,2997.0,45.34,94.35
1,2,191.0,2040.0,10.79,55.52
2,3,1003.0,3360.0,61.75,115.74
3,4,390.0,3322.0,24.73,91.83
4,7,208.0,2385.0,16.31,53.05
...,...,...,...,...,...
168,256,1190.0,3465.0,67.47,108.65
169,272,795.0,2828.0,42.06,84.36
170,273,1244.0,3500.0,70.27,114.53
171,276,358.0,2578.0,20.15,72.98


### ajoutons les disponibilités alimentaires à notre table

In [21]:
# joingnons les populations et les disponibilités alimentaires
table_1=pd.merge(tab_pop,dispo_alim, on='Code_pays',how='right')
table_1.shape

(173, 10)

In [22]:
table_1.isnull().sum()

Code_pays                  0
Pays                       2
Région                     2
Code ISO3                  2
pop_17                     2
pop_18                     2
Dispo_alim_Animaux         0
Dispo_alim_general         0
Dispo_protéines_Animaux    0
Dispo_protéines_general    0
dtype: int64

In [23]:
table_1[table_1.isna().any(axis=1)]

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general
26,41,NaN,NaN,NaN,NaN,NaN,739.0,3206.0,40.59,101.66
172,351,NaN,NaN,NaN,NaN,NaN,741.0,3203.0,40.92,101.55


### Etudions les données de la fao sur le prix du poulet en 2018

In [24]:
prix_poulet.sample()

,Code Domaine,Domaine,Code zone (FAO),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Code Mois,Mois,Unité,Valeur,Symbole,Description du Symbole
7,PP,Prix à la Production,80,Bosnie-Herzégovine,5532,Prix à la Production (USD/tonne),1095,"Viande poids vif, poulet",2018,2018,7021,Valeur annuel,USD,714.2,Fc,Donnée calculée


In [25]:
# sélectionnons les colonnes utiles
prix_poulet=prix_poulet[['Code zone (FAO)', 'Produit','Valeur']]

In [26]:
prix_poulet

,Code zone (FAO),Produit,Valeur
0,3,"Viande poids vif, poulet",2829.8
1,79,"Viande poids vif, poulet",1020.3
2,9,"Viande poids vif, poulet",948.2
3,1,"Viande poids vif, poulet",819.5
4,11,"Viande poids vif, poulet",1244.7
...,...,...,...
60,216,"Viande poids vif, poulet",1064.7
61,223,"Viande poids vif, poulet",2640.5
62,230,"Viande poids vif, poulet",1062.6
63,234,"Viande poids vif, poulet",1334.4


In [27]:
### Mettons le produit en colonne par un pivot
prix_poulet= prix_poulet.pivot_table(index='Code zone (FAO)',columns='Produit', values='Valeur')
prix_poulet.reset_index(inplace=True)
#renommons nos colonnes
prix_poulet.columns=['Code_pays','cout_prod_anuel_poulet',]
prix_poulet

,Code_pays,cout_prod_anuel_poulet
0,1,819.5
1,3,2829.8
2,9,948.2
3,11,1244.7
4,26,2594.7
...,...,...
60,255,1030.1
61,256,3977.3
62,272,1045.3
63,299,2498.9


### ajoutons le coût de production aux données

In [28]:
# joignons nos données
table_2=pd.merge(table_1,prix_poulet, on ='Code_pays', how ='left')
table_2.isnull().sum()

Code_pays                    0
Pays                         2
Région                       2
Code ISO3                    2
pop_17                       2
pop_18                       2
Dispo_alim_Animaux           0
Dispo_alim_general           0
Dispo_protéines_Animaux      0
Dispo_protéines_general      0
cout_prod_anuel_poulet     110
dtype: int64

In [29]:
table_2[table_2.isnull().any(axis=1)]

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet
1,2,Afghanistan,Asie,AFG,36296113.0,37171921.0,191.0,2040.0,10.79,55.52,NaN
3,4,Algérie,Afrique,DZA,41389189.0,42228408.0,390.0,3322.0,24.73,91.83,NaN
4,7,Angola,Afrique,AGO,29816766.0,30809787.0,208.0,2385.0,16.31,53.05,NaN
5,8,Antigua-et-Barbuda,Amerique,ATG,95426.0,96286.0,740.0,2445.0,51.74,80.19,NaN
7,10,Australie,Oceanie,AUS,24584620.0,24898152.0,1051.0,3391.0,70.70,105.94,NaN
...,...,...,...,...,...,...,...,...,...,...,...
165,249,Yémen,Asie,YEM,27834819.0,28498683.0,142.0,2154.0,10.38,54.52,NaN
166,251,Zambie,Afrique,ZMB,16853599.0,17351708.0,159.0,2002.0,13.70,60.53,NaN
170,273,Monténégro,Europe,MNE,627563.0,627809.0,1244.0,3500.0,70.27,114.53,NaN
171,276,Soudan,Afrique,SDN,40813397.0,41801533.0,358.0,2578.0,20.15,72.98,NaN


###  Données sur la disponibilité intérieure des volailles

In [30]:
# affichonns la table
volailles.sample()

,Code Domaine,Domaine,Code zone (FAO),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole
283,FBS,Bilans Alimentaires (2014-),256,Luxembourg,5301,Disponibilité intérieure,2734,Viande de Volailles,2018,2018,Milliers de tonnes,11,S,Données standardisées


In [31]:
volailles['Élément'].unique()

array(['Production', 'Importations - Quantité',
       'Disponibilité intérieure'], dtype=object)

In [32]:
# sélectionnons les colonnes
volailles=volailles[['Code zone (FAO)','Élément','Valeur']]

In [33]:
# afichons les labels de la colonne élement comme des colonnes

volailles=volailles.pivot_table(index='Code zone (FAO)',columns='Élément',values='Valeur')
volailles.reset_index(inplace=True)

# renommons nos colonnes
volailles.columns=['Code_pays','Disponibilité_intérieure','Importations','Production']
volailles.sample()

,Code_pays,Disponibilité_intérieure,Importations,Production
36,56,400.0,57.0,349.0


### ajoutons les données des disponibités alimentaires aux autres informations

In [34]:
# joingnons nos table
table_3=pd.merge(table_2,volailles, on ='Code_pays', how ='left')
table_3.shape

(173, 14)

### travaillons sur les données d'Indicateur économique : Pouvoir d'achat par habitant (PIB/habitant)
Nous avons le pib sur l'année en cours (2018) ainsi que celui de l'année précédente (2017)

In [35]:
#données sur le PIB
pib.sample()

,Code Domaine,Domaine,Code zone (FAO),Zone,Code Élément,Élément,Code Produit,Produit,Code année,Année,Unité,Valeur,Symbole,Description du Symbole,Note
79,MK,Indicateurs macro,351,Chine,6119,Valeur US $ par habitant,22008,Produit Intérieur Brut,2018,2018,US$,9968.778214,Fc,Donnée calculée,NaN


In [36]:
# sélectionnons nos colonnes
pib=pib[['Code zone (FAO)','Produit','Année','Valeur']]

#mettons le produit en colonnes par un pivot_table
pib=pib.pivot_table(index='Code zone (FAO)',columns='Année', values='Valeur')
pib.reset_index(inplace=True)

# renommons nos colonnes
pib.columns=['Code_pays','pib_17_hbt','pib_18_hbt']
pib.sample()

,Code_pays,pib_17_hbt,pib_18_hbt
21,25,1911.431453,2000.523017


### ajoutons le PIB/hbt à nos autres données

In [37]:
table =pd.merge(table_3,pib, on ='Code_pays', how ='left')
table.shape

(173, 16)

In [38]:
table.dtypes

Code_pays                     int64
Pays                         object
Région                       object
Code ISO3                    object
pop_17                      float64
pop_18                      float64
Dispo_alim_Animaux          float64
Dispo_alim_general          float64
Dispo_protéines_Animaux     float64
Dispo_protéines_general     float64
cout_prod_anuel_poulet      float64
Disponibilité_intérieure    float64
Importations                float64
Production                  float64
pib_17_hbt                  float64
pib_18_hbt                  float64
dtype: object

### ajoutons le distance des pays à la France à nos autres données

In [39]:
table = pd.merge(table,distances_france_km[['Code ISO3','distance_france']],on= 'Code ISO3', how = 'left')

## NETTOYONS NOS DONNES

 ## - Supprimons la chine et conservons ses subdivions pour avoir une finesse de données

In [40]:

table.drop(table.loc[table['Pays']=='Chine'].index, inplace = True)

In [41]:
# vérification de la supression
table[table['Pays']=='Chine']

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france


## -  Supprimons également les très petits pays.
* Il s'agit de ceux qui ont une faible population < 500000 hbts

In [42]:
# liste des pays ayant une population < 500 000
petit=table[table['pop_18'] < 500000]
petit.sample()

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france
64,99,Islande,Europe,ISL,334393.0,336713.0,1651.0,3654.0,104.98,146.13,NaN,10.0,1.0,9.0,73231.725965,76437.780317,2235.750569


In [43]:
table.shape[0]

173

In [44]:
# Proportion des très petits pays
prop_petit_pays=(petit.shape[0]/table.shape[0])*100
print ("les très petits pays (ceux dont la population <500000) représentent {} % de notre  échantillon de pays, nous les supprimons.".format(round(prop_petit_pays, 2)))

les très petits pays (ceux dont la population <500000) représentent 9.83 % de notre  échantillon de pays, nous les supprimons.


In [45]:
# prenons uniquement les pays dont la population est >=500 000

table = table[table['pop_18'] >= 500000 ]
table.sample()

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france
112,159,Nigéria,Afrique,NGA,190873244.0,195874683.0,84.0,2572.0,7.34,58.62,NaN,211.0,18.0,193.0,1968.687237,2153.526494,4700.791412


## Supprimons également la corré du nord
* Très peu réputé pour son libre échange 
* Contient près trop de valeurs manquantes

In [46]:
table[table['Pays']=='République populaire démocratique de Corée']

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france
80,116,République populaire démocratique de Corée,Asie,PRK,25429825.0,25549604.0,129.0,2019.0,10.2,53.34,NaN,43.0,0.0,45.0,686.340258,687.943178,NaN


In [47]:
# supression de la Corée du nord
table.drop(table.loc[table['Pays']=='République populaire démocratique de Corée'].index, inplace = True)


In [48]:
#table[table['pop_18']]
table = table.copy()

In [49]:
table.sample()

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france
102,146,République de Moldova,Europe,MDA,4059684.0,4051944.0,597.0,2383.0,30.27,61.64,1159.9,76.0,19.0,46.0,2381.895178,2827.641046,1981.378059


# Calcul des variables pour l'analyse
<br/>

## - La variation de la population au cours d'une année (2017 - 2018)

In [50]:
# Différence de population entre l'année 2018 et l'année 2017 en pourcentage
# Nous nommerons cette variable "evol_pop"
table['evol_pop'] = ((table['pop_18'] - table['pop_17']) / table['pop_17']) * 100
table.sample()

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france,evol_pop
83,119,Lettonie,Europe,LVA,1951097.0,1928459.0,1029.0,3229.0,63.08,102.72,1447.8,41.0,47.0,34.0,15611.095187,17846.380379,1707.614663,-1.16027


In [51]:
table

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,cout_prod_anuel_poulet,Disponibilité_intérieure,Importations,Production,pib_17_hbt,pib_18_hbt,distance_france,evol_pop
0,1,Arménie,Asie,ARM,2944791.0,2951745.0,795.0,2997.0,45.34,94.35,819.5,45.0,33.0,12.0,3914.525246,4220.534194,3439.396932,0.236146
1,2,Afghanistan,Asie,AFG,36296113.0,37171921.0,191.0,2040.0,10.79,55.52,NaN,53.0,25.0,29.0,513.085978,483.885875,5594.223377,2.412953
2,3,Albanie,Europe,ALB,2884169.0,2882740.0,1003.0,3360.0,61.75,115.74,2829.8,27.0,12.0,16.0,4514.204908,5254.384798,1603.944535,-0.049546
3,4,Algérie,Afrique,DZA,41389189.0,42228408.0,390.0,3322.0,24.73,91.83,NaN,295.0,0.0,295.0,4109.701336,4153.957220,1345.460397,2.027629
4,7,Angola,Afrique,AGO,29816766.0,30809787.0,208.0,2385.0,16.31,53.05,NaN,362.0,335.0,27.0,4095.811686,3289.643374,6478.953877,3.330411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,255,Belgique,Europe,BEL,11419748.0,11482178.0,1147.0,3769.0,58.58,99.87,1030.1,162.0,303.0,470.0,44020.066762,47354.636623,264.600808,0.546685
168,256,Luxembourg,Europe,LUX,591910.0,604245.0,1190.0,3465.0,67.47,108.65,3977.3,11.0,12.0,0.0,108431.931751,117369.540527,287.972186,2.083932
169,272,Serbie,Europe,SRB,8829628.0,8802754.0,795.0,2828.0,42.06,84.36,1045.3,103.0,17.0,96.0,6275.194450,7214.957742,1449.575080,-0.304362
170,273,Monténégro,Europe,MNE,627563.0,627809.0,1244.0,3500.0,70.27,114.53,NaN,10.0,9.0,4.0,7738.844301,8771.631140,1494.182007,0.039199


## - Proportion de protéine d'origine animale

In [52]:
# proportion de protéines d'origine animale par rapport à la quantité totale de protéines dans la disponibilité alimentaire du pays
# Nous la nommerons "prop_prot_anim"
table['prop_prot_anim']=(table['Dispo_protéines_Animaux']/table['Dispo_protéines_general'])*100

## - Disponibilité alimentaire en protéine par habitant

In [53]:
# Nommons la disponibilité alimentaire en protéine par habitant "dispo_alim_prot_hbt"
table['dispo_alim_prot_hbt']=table['Dispo_protéines_general']*365

## - Disponibilité alimentaire en calories par habitant

In [54]:
# Nommons la disponibilité alimentaire en calories par habitant "dispo_alim_cal_hbt"
table['dispo_alim_cal_hbt']=(table['Dispo_alim_general']*365)*1000

## - Quantité de volailles importée pour chaque habitant

In [55]:
# nommons cela "import_volaille_kg_hab_an"
table['import_volaille_kg_hab_an']=(table['Importations']*1000000)/table['pop_18']

## - Taux de dépendance aux importations

Ce taux est défini en économie comme le rapport entre les importations et la disponibilité intérieure du pays

In [56]:
# Nous nommerons cela "taux_dep_imp"
table['taux_dep_imp']=table['Importations']/table['Disponibilité_intérieure']*100

* Nous pouvons aussi calculer le 
## - Taux d'autosuffisance

Il exprime l'importance de la production, par rapport à la disponibilité intérieure du pays

In [57]:
# Nous nommerons cela "taux_autosuffi"
table['taux_autosuffi']=table['Production']/table['Disponibilité_intérieure']*100


## - Quantité de volailles produites / habitant

In [58]:
table['production_volaille_kg_hab']=(table['Production']*1000000)/table['pop_18']

## - Evolution du pouvoir d'achat(pib) aucours de ces deux années

In [59]:
table['proportion_pib']=((table['pib_18_hbt']-table['pib_17_hbt'])/table['pib_17_hbt'])*100

In [60]:
# Affichons notre table finale
table.head()

,Code_pays,Pays,Région,Code ISO3,pop_17,pop_18,Dispo_alim_Animaux,Dispo_alim_general,Dispo_protéines_Animaux,Dispo_protéines_general,...,distance_france,evol_pop,prop_prot_anim,dispo_alim_prot_hbt,dispo_alim_cal_hbt,import_volaille_kg_hab_an,taux_dep_imp,taux_autosuffi,production_volaille_kg_hab,proportion_pib
0,1,Arménie,Asie,ARM,2944791.0,2951745.0,795.0,2997.0,45.34,94.35,...,3439.396932,0.236146,48.055114,34437.75,1.093905e+09,11.179828,73.333333,26.666667,4.065392,7.817268
1,2,Afghanistan,Asie,AFG,36296113.0,37171921.0,191.0,2040.0,10.79,55.52,...,5594.223377,2.412953,19.434438,20264.80,7.446000e+08,0.672551,47.169811,54.716981,0.780159,-5.691074
2,3,Albanie,Europe,ALB,2884169.0,2882740.0,1003.0,3360.0,61.75,115.74,...,1603.944535,-0.049546,53.352341,42245.10,1.226400e+09,4.162706,44.444444,59.259259,5.550275,16.396683
3,4,Algérie,Afrique,DZA,41389189.0,42228408.0,390.0,3322.0,24.73,91.83,...,1345.460397,2.027629,26.930197,33517.95,1.212530e+09,0.000000,0.000000,100.000000,6.985819,1.076864
4,7,Angola,Afrique,AGO,29816766.0,30809787.0,208.0,2385.0,16.31,53.05,...,6478.953877,3.330411,30.744581,19363.25,8.705250e+08,10.873168,92.541436,7.458564,0.876345,-19.682748


In [61]:
# affichons le nombre de colonnes
table.columns

Index(['Code_pays', 'Pays', 'Région', 'Code ISO3', 'pop_17', 'pop_18',
       'Dispo_alim_Animaux', 'Dispo_alim_general', 'Dispo_protéines_Animaux',
       'Dispo_protéines_general', 'cout_prod_anuel_poulet',
       'Disponibilité_intérieure', 'Importations', 'Production', 'pib_17_hbt',
       'pib_18_hbt', 'distance_france', 'evol_pop', 'prop_prot_anim',
       'dispo_alim_prot_hbt', 'dispo_alim_cal_hbt',
       'import_volaille_kg_hab_an', 'taux_dep_imp', 'taux_autosuffi',
       'production_volaille_kg_hab', 'proportion_pib'],
      dtype='object')

In [62]:
table.shape

(153, 26)

In [63]:
# sélectionnons les colonnes variables
table=table[['Pays','pop_18','Région','Code ISO3','evol_pop','distance_france','Dispo_alim_general','Dispo_protéines_general','prop_prot_anim','dispo_alim_prot_hbt','dispo_alim_cal_hbt','taux_autosuffi','taux_dep_imp','pib_18_hbt','cout_prod_anuel_poulet']]

# Exportons la table des données finale obtenu pour notre analyse exploratoire

In [64]:
table.head(3)

,Pays,pop_18,Région,Code ISO3,evol_pop,distance_france,Dispo_alim_general,Dispo_protéines_general,prop_prot_anim,dispo_alim_prot_hbt,dispo_alim_cal_hbt,taux_autosuffi,taux_dep_imp,pib_18_hbt,cout_prod_anuel_poulet
0,Arménie,2951745.0,Asie,ARM,0.236146,3439.396932,2997.0,94.35,48.055114,34437.75,1.093905e+09,26.666667,73.333333,4220.534194,819.5
1,Afghanistan,37171921.0,Asie,AFG,2.412953,5594.223377,2040.0,55.52,19.434438,20264.80,7.446000e+08,54.716981,47.169811,483.885875,NaN
2,Albanie,2882740.0,Europe,ALB,-0.049546,1603.944535,3360.0,115.74,53.352341,42245.10,1.226400e+09,59.259259,44.444444,5254.384798,2829.8


In [65]:
# exportons la table finale
table.to_csv('dataset_p5/new_data/donnees_finales.csv')